In [ ]:
! pip install langchain_core langchain langchain_cohere langchain-community langchain-chroma duckduckgo-search langgraph

In [ ]:
# import
from langchain_chroma import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain_cohere import CohereEmbeddings
from langchain_community.llms import Cohere

llm = Cohere(model="command-r-plus", max_tokens=1024, temperature=0.2)

def Retriever(folder_path,chunk_size=1200, chunk_overlap=200):
    # Load documents from the specified folder
    loader = PyPDFDirectoryLoader(folder_path)
    documents = loader.load()

    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size , chunk_overlap=chunk_overlap)
    chunks = text_splitter.split_documents(documents)

    embeddings = CohereEmbeddings(model="embed-english-light-v3.0")

    # Create a Chroma vector store from the document chunks and embeddings
    db = Chroma.from_documents(chunks, embeddings)
    return db.as_retriever()

# Initialize the Retriever with the path to your PDF data source
retriever = Retriever(file_name="your_data_source-folder",chunk_size=1200, chunk_overlap=200)


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `Cohere` was deprecated in LangChain 0.1.14 and will be removed in 0.3.0. An updated version of the class exists in the langchain-cohere package and should be used instead. To use it run `pip install -U langchain-cohere` and import as `from langchain_cohere import Cohere`.
  warn_deprecated(


In [ ]:
retriever.invoke("What is MOA ? ")

In [ ]:
from typing import Literal

from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.output_parsers import StructuredOutputParser, PydanticOutputParser

# Data model
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal["vectorstore", "websearch"] = Field(
        ...,
        description="Given a user question choose to route it to web search or a vectorstore.",
    )

# Output parser
routing_parser = PydanticOutputParser(pydantic_object=RouteQuery)

# Prompt
routing_system_template = """You are an expert at routing a user question to a vectorstore or websearch.
The vectorstore contains documents related to new AI Method called Mixture-of-Agents (MoA) .
Use the vectorstore for questions on these topics. For all else, use websearch."""
routing_system_message_prompt = SystemMessagePromptTemplate.from_template(routing_system_template)

routing_human_template = "{question}\n\n{format_instructions}"
routing_human_message_prompt = HumanMessagePromptTemplate.from_template(routing_human_template)

# Combine prompts and add formatting instructions for structured output
chat_prompt = ChatPromptTemplate.from_messages(
    [routing_system_message_prompt, routing_human_message_prompt]
)

# Format instructions for the LLM
routing_format_instructions = routing_parser.get_format_instructions()

# Create a chain to generate and parse the response
routing_chain = chat_prompt | llm | routing_parser

# Test the chain
print(routing_chain.invoke({"question": "What is MOA ? ", "format_instructions": routing_format_instructions}))
print(routing_chain.invoke({"question": "Who is barak obama ?", "format_instructions": routing_format_instructions}))

datasource='vectorstore'
datasource='websearch'


In [ ]:
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser

# Data model
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""
    binary_score: Literal["yes", "no"] = Field(description="Documents are relevant to the question, 'yes' or 'no'")

# Output parser
relevance_parser = PydanticOutputParser(pydantic_object=GradeDocuments)



# System prompt
relevance_system_template = """You are a grader assessing relevance of a retrieved document to a user question.
If the document contains keyword(s) or semantic meaning related to the question, grade it as relevant.
Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""
relevance_system_message_prompt = SystemMessagePromptTemplate.from_template(relevance_system_template)

# Human prompt
relevance_human_template = "Retrieved document: \n\n {document} \n\n User question: {question}\n\n{format_instructions}"
relevance_human_message_prompt = HumanMessagePromptTemplate.from_template(relevance_human_template)

# Combine prompts into a chat prompt template
chat_prompt = ChatPromptTemplate.from_messages(
    [relevance_system_message_prompt, relevance_human_message_prompt]
)

# Format instructions for the LLM
retrieval_format_instructions = relevance_parser.get_format_instructions()

# Create a chain to generate and parse the response
retrieval_grader_relevance = chat_prompt | llm | relevance_parser

# Example inputs (replace with actual retrieval and LLM instances)
question = "What is MOA ? "
docs = retriever.invoke(question)  # Assuming `retriever` is your document retriever instance
doc_txt = docs[1].page_content  # Access the content of the second retrieved document

# Test the chain
result = retrieval_grader_relevance.invoke({
    "question": question,
    "document": doc_txt,
    "format_instructions": retrieval_format_instructions
})

print(result)


binary_score='yes'


In [ ]:
### Generate

from langchain_core.output_parsers import StrOutputParser

# Prompt
prompt = ChatPromptTemplate.from_template(
    """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:"""
)

# Chain
rag_chain = prompt | llm | StrOutputParser()

# Run
generation = rag_chain.invoke({"context": docs, "question": question})
print(generation)

Mixture-of-Agents is a layered architecture comprised of LLM agents, where each layer's agents use outputs from the previous layer to generate responses.


In [ ]:
from typing import Literal
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.output_parsers import StructuredOutputParser, PydanticOutputParser

# Data model
class GradeHallucinations(BaseModel):
    """Binary score for hallucination present in generation answer."""
    binary_score: Literal["yes", "no"] = Field(...,description="Don't consider calling external APIs for additional information. Answer is supported by the facts, 'yes' or 'no'.")

# Output parser
grader_parser = PydanticOutputParser(pydantic_object=GradeHallucinations)

# System prompt
grader_system_template = """You are a grader assessing whether an LLM generation is supported by a set of retrieved facts.
Restrict yourself to give a binary score, either 'yes' or 'no'. If the answer is supported or partially supported by the set of facts, consider it a yes.
Don't consider calling external APIs for additional information as consistent with the facts."""
grader_system_message_prompt = SystemMessagePromptTemplate.from_template(grader_system_template)

# Human prompt
grader_human_template = "Set of facts: \n\n {documents} \n\n LLM generation: {generation}\n\n{format_instructions}"
grader_human_message_prompt = HumanMessagePromptTemplate.from_template(grader_human_template)

# Combine prompts into a chat prompt template
grader_chat_prompt = ChatPromptTemplate.from_messages(
    [grader_system_message_prompt, grader_human_message_prompt]
)

# Format instructions for the LLM
grader_format_instructions = grader_parser.get_format_instructions()

# Create a chain to generate and parse the response
hallucination_grader = grader_chat_prompt | llm | grader_parser


# Test the chain
result = hallucination_grader.invoke({
    "documents": docs,
    "generation": generation,
    "format_instructions": grader_format_instructions
})

print(result)


binary_score='yes'


In [ ]:
# Data model
class GradeAnswer(BaseModel):
    """Binary score to assess if the answer addresses the question."""
    binary_score: Literal["yes", "no"] = Field(description="Answer addresses the question, 'yes' or 'no'")

# Output parser
answer_grader_parser = PydanticOutputParser(pydantic_object=GradeAnswer)

# System prompt
answer_grader_system_template = """You are a grader assessing whether an answer addresses / resolves a question.
Give a binary score 'yes' or 'no'. 'Yes' means that the answer resolves the question."""
answer_grader_system_message_prompt = SystemMessagePromptTemplate.from_template(answer_grader_system_template)

# Human prompt
answer_grader_human_template = "User question: \n\n {question} \n\n LLM generation: {generation}\n\n{format_instructions}"
answer_grader_human_message_prompt = HumanMessagePromptTemplate.from_template(answer_grader_human_template)

# Combine prompts into a chat prompt template
chat_prompt = ChatPromptTemplate.from_messages(
    [answer_grader_system_message_prompt, answer_grader_human_message_prompt]
)

# Format instructions for the LLM
answer_format_instructions = answer_grader_parser.get_format_instructions()

# Create a chain to generate and parse the response
answer_grader = chat_prompt | llm | answer_grader_parser

# Test the chain
result = answer_grader.invoke({
    "question": question,
    "generation": generation,
    "format_instructions": answer_format_instructions
})

print(result)

binary_score='yes'


In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun

duckduckgo_search = DuckDuckGoSearchRun()

In [ ]:
from typing_extensions import TypedDict
from typing import List

class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        web_search: whether to add search
        documents: list of documents
    """
    question : str
    generation : str
    web_search : str
    documents : List[str]

In [ ]:
from langchain.schema import Document

### Nodes

def retrieve(state):
    """
    Retrieve documents from vectorstore

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE from Vector Store DB---")
    question = state["question"]

    # Retrieval
    documents = retriever.invoke(question)
    return {"documents": documents, "question": question}

In [ ]:
def generate(state):
    """
    Generate answer using RAG on retrieved documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE Answer---")
    question = state["question"]
    documents = state["documents"]

    # RAG generation
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation}

In [ ]:
def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question
    If any document is not relevant, we will set a flag to run web search

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Filtered out irrelevant documents and updated web_search state
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]
    # Score each doc
    filtered_docs = []
    web_search = "No"
    for d in documents:
        score = retrieval_grader_relevance.invoke({"question": question, "document": d.page_content , "format_instructions" : retrieval_format_instructions})
        grade = score.binary_score
        # Document relevant
        if grade.lower() == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        # Document not relevant
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            # We do not include the document in filtered_docs
            # We set a flag to indicate that we want to run web search
            web_search = "Yes"
            continue
    return {"documents": filtered_docs, "question": question, "web_search": web_search}

In [ ]:
def web_search(state):
    """
    Web search based based on the question

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Appended web results to documents
    """

    print("---WEB SEARCH. Append to vector store db---")
    question = state["question"]
    documents = state["documents"]

    # Web search
    docs = duckduckgo_search.invoke({"query": question})
    web_results = Document(page_content=docs)
    if documents is not None:
        documents.append(web_results)
    else:
        documents = [web_results]
    return {"documents": documents, "question": question}

In [ ]:
def route_question(state):
    """
    Route question to web search or RAG

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """

    print("---ROUTE QUESTION---")
    question = state["question"]

    source = routing_chain.invoke({"question": question , "format_instructions" : routing_format_instructions})
    if source.datasource == 'websearch':
        print("---ROUTE QUESTION TO WEB SEARCH---")
        return "websearch"
    elif source.datasource == 'vectorstore':
        print("---ROUTE QUESTION TO RAG---")
        return "vectorstore"


In [ ]:
def decide_to_generate(state):
    """
    Determines whether to generate an answer, or add web search

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    question = state["question"]
    web_search = state["web_search"]
    filtered_documents = state["documents"]

    if web_search == "Yes":
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print("---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, INCLUDE WEB SEARCH---")
        return "websearch"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE---")
        return "generate"


In [ ]:
def grade_generation_v_documents_and_question(state):
    """
    Determines whether the generation is grounded in the document and answers question

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """

    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]

    score = hallucination_grader.invoke({"documents": documents, "generation": generation , "format_instructions"  : grader_format_instructions})
    grade = score.binary_score

    # Check hallucination
    if grade == "yes":
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        # Check question-answering
        print("---GRADE GENERATION vs QUESTION---")
        score = answer_grader.invoke({"question": question,"generation": generation , "format_instructions" : answer_format_instructions})
        grade = score.binary_score
        if grade == "yes":
            print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        else:
            print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
            return "not useful"
    else:
        print("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not supported"

In [ ]:
from langgraph.graph import END, StateGraph

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("websearch", web_search) # web search # key: action to do
workflow.add_node("retrieve", retrieve) # retrieve
workflow.add_node("grade_documents", grade_documents) # grade documents
workflow.add_node("generate", generate) # generatae

workflow.add_edge("websearch", "generate") #start -> end of node
workflow.add_edge("retrieve", "grade_documents")

# Build graph
workflow.set_conditional_entry_point(
    route_question,
    {
        "websearch": "websearch",
        "vectorstore": "retrieve",
    },
)

workflow.add_conditional_edges(
    "grade_documents", # start: node
    decide_to_generate, # defined function
    {
        "websearch": "websearch", #returns of the function
        "generate": "generate",   #returns of the function
    },
)
workflow.add_conditional_edges(
    "generate", # start: node
    grade_generation_v_documents_and_question, # defined function
    {
        "not supported": "generate", #returns of the function
        "useful": END,               #returns of the function
        "not useful": "websearch",   #returns of the function
    },
)

# Compile
app = workflow.compile()

In [ ]:
from pprint import pprint
inputs = {"question": "what is Mixture-of-Agents (MoA) ? "}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Finished running: {key}:")
pprint(value["generation"])

---ROUTE QUESTION---
---ROUTE QUESTION TO RAG---
---RETRIEVE from Vector Store DB---
'Finished running: retrieve:'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
'Finished running: grade_documents:'
---GENERATE Answer---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---
'Finished running: generate:'
('Mixture-of-Agents (MoA) is a methodology that utilizes multiple LLMs to '
 'iteratively synthesize and refine responses, improving overall output '
 'quality. MoA enhances collaborative potential by introducing additional '
 'aggregators to build upon outputs from other models. This approach has shown '
 'substantial improvements in response quality over relying on a single model.')


In [ ]:
from pprint import pprint
inputs = {"question": "Who is barak obama ?"}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Finished running: {key}:")
pprint(value["generation"])

---ROUTE QUESTION---
---ROUTE QUESTION TO WEB SEARCH---
---WEB SEARCH. Append to vector store db---
'Finished running: websearch:'
---GENERATE Answer---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---
'Finished running: generate:'
('Barack Obama was the 44th President of the United States, serving from 2009 '
 'to 2017. He was the first African American to hold this office and is known '
 'for his charismatic personality and focus on political change. Obama '
 'previously served in the Illinois Senate and is the son of a Kenyan father '
 'and a white mother.')
